In [114]:
import onnx
from onnx import helper,TensorProto
from qonnx.core.modelwrapper import ModelWrapper
from qonnx.core.datatype import DataType
import numpy as np
from qonnx.util.basic import gen_finn_dt_tensor
import onnx.version_converter as vc
from onnx.backend.test.case.node import expect

model = ModelWrapper("tinyyolo-20210831.onnx")
# converted_model = vc.convert_version(model.model,12)
# model_new = ModelWrapper(converted_model)
# model_new.save("New_opset_model.onnx")


In [115]:
# shape node 
top_in = helper.make_tensor_value_info("top_in", TensorProto.FLOAT, [1,64,16,16])
shape_out = helper.make_tensor_value_info("735", TensorProto.INT64,[4])
shape_node = helper.make_node(
            "Shape",
            inputs = ["top_in"],
            outputs = ["735"])

In [119]:
# Slice node
input_shape = [4]
output_shape = [2]
param_shape = [1]
idt = DataType["UINT8"]
param_dt = DataType["UINT4"]
slice_in = helper.make_tensor_value_info("735", TensorProto.INT64, input_shape)
top_out = helper.make_tensor_value_info("top_out", TensorProto.INT64, output_shape)
slice_attr = {}
slice_attr["starts"] = [1]
slice_attr["ends"] = [1]
slice_attr["axes"] = [1]

value_info = [
        helper.make_tensor_value_info("starts", TensorProto.INT64, param_shape),
        helper.make_tensor_value_info("ends",   TensorProto.INT64, param_shape),
        helper.make_tensor_value_info("axes",   TensorProto.INT64, param_shape),
]

slice_node = helper.make_node(
            "Slice",
            inputs  = ["735"],#,"starts","ends","axes"],
            outputs = ["top_out"],
            **slice_attr
            )


In [120]:
graph = helper.make_graph(
    nodes = [shape_node,slice_node],
    name = "slice_graph",
    inputs = [top_in],
    outputs = [top_out],
#     value_info = value_info,
)

In [123]:
model_config = {}
model_config["opset_imports"] = [helper.make_operatorsetid("",9)]

modelProto = helper.make_model(graph,**model_config)
model = ModelWrapper(modelProto)


model.set_tensor_datatype("starts", param_dt)
model.set_initializer("starts",gen_finn_dt_tensor(param_dt,param_shape))
model.set_tensor_datatype("ends", param_dt)
model.set_initializer("ends",gen_finn_dt_tensor(param_dt,param_shape))
model.set_tensor_datatype("axes", param_dt)
model.set_initializer("axes",gen_finn_dt_tensor(param_dt,param_shape))

converted_model = vc.convert_version(model.model,12)
model = ModelWrapper(converted_model)
model.save("slice.onnx")

In [122]:
import finn.builder.build_dataflow as build
import finn.builder.build_dataflow_config as build_cfg
import os
import shutil

model_file = "slice.onnx"

final_output_dir = "output_files/slice"

#Delete previous run results if exist
if os.path.exists(final_output_dir):
    shutil.rmtree(final_output_dir)
    print("Previous run results deleted!")

cfg = build.DataflowBuildConfig(
    output_dir          = final_output_dir,
    mvau_wwidth_max     = 80,
    target_fps          = 1000000,
    synth_clk_period_ns = 10.0,
    board               = "KV260_SOM",
#     steps               = "estimate_only_dataflow_steps",
    shell_flow_type     = build_cfg.ShellFlowType.VIVADO_ZYNQ,
    generate_outputs=[
        build_cfg.DataflowOutputType.BITFILE,
        build_cfg.DataflowOutputType.PYNQ_DRIVER,
        build_cfg.DataflowOutputType.DEPLOYMENT_PACKAGE,
    ]
)

Previous run results deleted!


In [124]:
%%time
build.build_dataflow_cfg(model_file, cfg)

Traceback (most recent call last):
  File "/workspace/finn/src/finn/builder/build_dataflow.py", line 166, in build_dataflow_cfg
    model = transform_step(model, cfg)
  File "/workspace/finn/src/finn/builder/build_dataflow_steps.py", line 243, in step_tidy_up
    model = model.transform(FoldConstants())
  File "/workspace/qonnx/src/qonnx/core/modelwrapper.py", line 140, in transform
    (transformed_model, model_was_changed) = transformation.apply(transformed_model)
  File "/workspace/qonnx/src/qonnx/transformation/fold_constants.py", line 53, in apply
    ishape = model.get_tensor_shape(n.input[0])
IndexError: list index (0) out of range


Building dataflow accelerator from slice.onnx
Intermediate outputs will be generated in /workspace/results
Final outputs will be generated in output_files/slice
Build log is at output_files/slice/build_dataflow.log
Running step: step_qonnx_to_finn [1/17]
Running step: step_tidy_up [2/17]
> /workspace/qonnx/src/qonnx/transformation/fold_constants.py(53)apply()
     51             node_out = n.output[0]
     52             is_all_constant_inputs = len(node_inp_dyn) == 0
---> 53             ishape = model.get_tensor_shape(n.input[0])
     54             is_const_shape = (n.op_type == "Shape") and (ishape is not None)
     55             exclude = n.op_type in self.exclude_op_types

--KeyboardInterrupt--

KeyboardInterrupt: Interrupted by user
Build failed
CPU times: user 368 ms, sys: 65.8 ms, total: 434 ms
Wall time: 24.5 s


-1